## For process the raw data collected from Mturk

input: main_batches.csv; initial_batches.csv \\\
generate: all_processed

In [1]:

import pandas as pd
import ast
import json
import re
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import random
import traceback
import html #for unescape & < >

import emoji
import os



In [3]:

data_3000=pd.read_csv('main_batches.csv')
ppdict = {n: grp.loc[n].to_dict('index') for n, grp in data_3000.set_index(['InputId', 'WorkerId']).groupby(level='InputId')}

new_dict={}
for tweet_id,collection in ppdict.items():
    basic_info=list(collection.values())[0]
    # seperate strings but keep delimiters
    txt = html.unescape(basic_info['Input.tweet'])
    
    temp = list(filter(None, re.split('([,.!?:()[\]"\s+])', txt)))
    # remove space strings from list and convert into np array
    tweet_split = np.array(list(filter(str.strip, temp)))
    #     tweet_split = np.array(re.split('[,.!?:()[\]"\s+]', basic_info['Input.tweet']))
#     gold_standard = basic_info['Input.prediction']
    workerids = list(collection.keys())
    
    sentence_score = []
    entity_label = []
    related_label = []
    workertime = []
    assignmentID = []
    for workerid,record in collection.items():
        sentence_score.append(record['Answer.optradio'])
        workertime.append(record['WorkTimeInSeconds'])
        token_labels = np.array(['O']*len(tweet_split),dtype=np.dtype(('U',10)))
        relation_lables = np.array([0]*len(tweet_split))
        assignmentID.append(record['AssignmentId'])
        try:
            if record['Answer.related_index'] != '[]' :
    #             print(record['Answer.related_index'])
    #             print('undefined' in record['Answer.related_index'])
                relation_lables_idx_str = sum([i.split(' ') for i in ast.literal_eval(record['Answer.related_index'])],[])

                relation_lables_idx = list(map(int, relation_lables_idx_str))
                relation_lables[relation_lables_idx] = 1

                
            if np.isnan(record['Answer.no-entity']) and re.split('[|]', record["Answer.html_output"])[1]!='': # the value is 1 when there is no entity to label
                html_output_list = ast.literal_eval(re.split('[|]', record["Answer.html_output"])[1])
                
                for e in html_output_list:
                    if 'idx' in list(e.keys()):

                        if ' ' in e['idx']:
                            idx = list(map(int, e['idx'].split(' ')))
                        else:
                            idx = ast.literal_eval(e['idx'])

                        if type(idx) is int:

#                             assert tweet_split[idx] == e['text']
                            token_labels[idx] = 'B-'+e['className'].split('-')[1]
                        else:
    #                         print(' '.join(tweet_split[idx]))
    #                         print(e['text'])
#                             if tweet_split[idx][0] != e['text'].split()[0] and  tweet_split[idx][-1] != e['text'].split()[-1]:
#                                 print(tweet_split[idx],e['text'])
#                             assert tweet_split[idx][0] == e['text'].split()[0] and  tweet_split[idx][-1] == e['text'].split()[-1]
                            idx=list(idx)
                            token_labels[idx[0]] = 'B-'+e['className'].split('-')[1]
                            token_labels[idx[1:]] = 'I-' + e['className'].split('-')[1]


                
        except Exception:
            traceback.print_exc()
            print('AssignmentId:',record['AssignmentId'],'Answer.related_index:',record['Answer.related_index'])

        entity_label.append(token_labels.tolist())
        related_label.append(relation_lables.tolist())
        
    new_dict[tweet_id]={'tweet':txt,'tweet_tokens':tweet_split.tolist(),
                       'workerid':workerids,'workertime':workertime,'sentence_score':sentence_score,
                        'entity_label':entity_label,'related_label':related_label,'assignmentID':assignmentID}



In [10]:
csvfile='initial_batch.csv'

data_200=pd.read_csv(csvfile)

# set tweet_id+worker_id as index and group by tweet_id 




# set tweet_id+worker_id as index and group by tweet_id 
ppdict = {n: grp.loc[n].to_dict('index') for n, grp in data_200.set_index(['InputId', 'WorkerId']).groupby(level='InputId')}

new_dict_i={}
for tweet_id,collection in ppdict.items():
    basic_info=list(collection.values())[0]
    # seperate strings but keep delimiters
    txt = html.unescape(basic_info['Input.tweet'])
    temp = list(filter(None, re.split('([,.!?:()[\]\\/"“”\s+])', txt)))
    # remove space strings from list and convert into np array
    tweet_split = np.array(list(filter(str.strip, temp)))
    #     tweet_split = np.array(re.split('[,.!?:()[\]"\s+]', basic_info['Input.tweet']))
#     gold_standard = basic_info['Input.prediction']
    workerids = list(collection.keys())
    
    sentence_score = []
    entity_label = []
    related_label = []
    workertime = []
    assignmentID = []
    for workerid,record in collection.items():
        sentence_score.append(record['Answer.optradio'])
        workertime.append(record['WorkTimeInSeconds'])
        token_labels = np.array(['O']*len(tweet_split),dtype=np.dtype(('U',10)))
        relation_lables = np.array([0]*len(tweet_split))
        assignmentID.append(record['AssignmentId'])
        try:
            if record['Answer.related_index'] != '[]' :
    #             print(record['Answer.related_index'])
    #             print('undefined' in record['Answer.related_index'])
                relation_lables_idx_str = sum([i.split(' ') for i in ast.literal_eval(record['Answer.related_index'])],[])

                relation_lables_idx = list(map(int, relation_lables_idx_str))
                relation_lables[relation_lables_idx] = 1

                
            if np.isnan(record['Answer.no-entity']) and re.split('[|]', record["Answer.html_output"])[1]!='': # the value is 1 when there is no entity to label
                html_output_list = ast.literal_eval(re.split('[|]', record["Answer.html_output"])[1])
                
                for e in html_output_list:
                    if 'idx' in list(e.keys()):

                        if ' ' in e['idx']:
                            idx = list(map(int, e['idx'].split(' ')))
                        else:
                            idx = ast.literal_eval(e['idx'])

                        if type(idx) is int:

                            assert tweet_split[idx] == e['text']
                            token_labels[idx] = 'B-'+e['className'].split('-')[1]
                        else:
    #                         print(' '.join(tweet_split[idx]))
    #                         print(e['text'])
#                             if tweet_split[idx][0] != e['text'].split()[0] and  tweet_split[idx][-1] != e['text'].split()[-1]:
#                                 print(tweet_split[idx],e['text'])
                            assert tweet_split[idx][0] == e['text'].split()[0] and  tweet_split[idx][-1] == e['text'].split()[-1]
                            idx=list(idx)
                            token_labels[idx[0]] = 'B-'+e['className'].split('-')[1]
                            token_labels[idx[1:]] = 'I-' + e['className'].split('-')[1]


                
        except Exception:
            traceback.print_exc()
            print('AssignmentId:',record['AssignmentId'],'Answer.related_index:',record['Answer.related_index'])

        entity_label.append(token_labels.tolist())
        related_label.append(relation_lables.tolist())
    new_dict_i[tweet_id]={'tweet':txt,'tweet_tokens':tweet_split.tolist(),
                       'workerid':workerids,'workertime':workertime,'sentence_score':sentence_score,
                        'entity_label':entity_label,'related_label':related_label,'assignmentID':assignmentID}



d_3000=pd.DataFrame.from_dict(new_dict).T
d_200=pd.DataFrame.from_dict(new_dict_i).T
d_all_processed=pd.concat([d_3000,d_200]).drop_duplicates(subset='tweet')

